# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv('../WeatherPy/Results/weather_data.csv')
weather_df.head()

,name,temp,humidity,clouds,wind_speed,country code,lat,lng
0,menongue,59.600,28.500,0.000,5.145,ao,-14.662,17.688
1,wawa,61.703,68.375,21.000,4.986,ca,48.000,-84.783
2,sainte-suzanne,67.719,75.417,56.042,18.461,re,-20.900,55.617
3,edson,64.175,62.583,26.500,7.240,ca,53.583,-116.417
4,fez,96.568,12.667,33.292,8.681,ma,34.037,-5.000


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key = g_key)

In [11]:
locations = weather_df[['lat', 'lng']]
weights_h = weather_df['humidity']
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights_h))
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

#### My Ideal Weather Conditions
* 75 degrees to 90 degrees
* clouds < 10
* Wind Speed < 1
* Humidity < 55

In [14]:
ideal = weather_df.loc[((weather_df['temp'] > 75) & 
                        (weather_df['temp'] > 90) &
                        (weather_df['clouds'] < 30 ) &
                        (weather_df['wind_speed'] < 10) &
                        (weather_df['humidity'] < 55)
                       )]
ideal

,name,temp,humidity,clouds,wind_speed,country code,lat,lng
29,south yuba city,92.571,28.708,11.875,6.938,us,39.117,-121.638
287,bhag,106.954,25.042,11.125,8.656,pk,29.041,67.824
306,esna,93.200,26.250,0.000,6.433,eg,25.293,32.553
376,arlit,93.809,24.333,7.375,5.055,ne,18.733,7.368
660,bam,99.400,8.375,18.042,9.228,ir,29.106,58.357
706,ridgecrest,100.824,13.333,26.208,7.580,us,35.623,-117.670
796,abu kamal,96.534,20.917,0.000,9.094,sy,34.450,40.919
823,tabas,101.275,11.875,0.000,4.419,ir,33.596,56.924
908,bafq,101.068,8.542,0.042,6.860,ir,31.613,55.411
918,sabha,95.286,15.542,0.000,9.355,ly,27.033,14.433


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [39]:
hotel_df = ideal.copy()
hotel_df.head()

,name,temp,humidity,clouds,wind_speed,country code,lat,lng
29,south yuba city,92.571,28.708,11.875,6.938,us,39.117,-121.638
287,bhag,106.954,25.042,11.125,8.656,pk,29.041,67.824
306,esna,93.200,26.250,0.000,6.433,eg,25.293,32.553
376,arlit,93.809,24.333,7.375,5.055,ne,18.733,7.368
660,bam,99.400,8.375,18.042,9.228,ir,29.106,58.357


In [47]:
hotel_names = []
hotel_ratings = []
for location in hotel_df.itertuples(index=False):
    #This will help us track whether we have found a hotel in each location before moving on to the next query
    vacay_not_found = True
    #perform search queries of hotels within 5,000 meters of my ideal locations.
    searchurl = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={location.lat},{location.lng}&radius=5000&type=hotel&key={g_key}&rankby=prominence'
    response = requests.get(searchurl).json()['results']
    iterator = 0 
    try:
        #This makes sure that while we have not found a hotel, we keep trying.
        while vacay_not_found:
            if ('name' in response[iterator].keys()) and ('rating' in response[iterator].keys()):
                hotel_names.append(response[iterator]['name'])
                hotel_ratings.append(response[iterator]['rating'])
                vacay_not_found = False
            else: 
                iterator += 1
    except Exception as e:
        print(e)
hotel_df['Hotel Name'] = hotel_names
hotel_df['Hotel Ratings'] = hotel_ratings
hotel_df.head()

,name,temp,humidity,clouds,wind_speed,country code,lat,lng,Hotel Name,Hotel Ratings
29,south yuba city,92.571,28.708,11.875,6.938,us,39.117,-121.638,Bonanza Inn & Suites Yuba City,3.9
287,bhag,106.954,25.042,11.125,8.656,pk,29.041,67.824,Raja Book Store,4.1
306,esna,93.200,26.250,0.000,6.433,eg,25.293,32.553,Orient Star Dahabyia Isna 05 Nights Each Sunday,4.4
376,arlit,93.809,24.333,7.375,5.055,ne,18.733,7.368,_yakkouza,3.7
660,bam,99.400,8.375,18.042,9.228,ir,29.106,58.357,شرکت بازرگانی مهد بازرگان جنوب,3.0


In [51]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{name}</dd>
<dt>Country</dt><dd>{country code}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lng"]]

In [53]:
# Add marker layer ontop of heat map
fig.add_layer(gmaps.marker_layer(locations, info_box_content = hotel_info))

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))